In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 14


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
4,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-10-05T22:58:24-05:00 -> 1696564704000, Until: 2023-10-06T22:58:24-05:00 -> 1696651104000
Group by Time: 30s


In [4]:
# PM2.5 Data Frame Sensors
df_pm25_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'pm25', GROUP_BY_TIME)
df_pm25_raw.head()
#df_pm25_raw.shape

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,,
2023-10-05 22:58:30-05:00,11.0,8.0,7.0,NaN,NaN,6.0,7.0,NaN,11.0,13.0,2.0,2.0,6.0,0.0
2023-10-05 22:59:00-05:00,10.0,7.0,7.0,NaN,NaN,6.0,5.0,NaN,10.0,13.0,2.0,3.0,6.0,0.0
2023-10-05 22:59:30-05:00,10.0,7.0,7.0,NaN,NaN,5.0,5.0,NaN,11.0,14.0,3.0,4.0,6.0,0.0
2023-10-05 23:00:00-05:00,10.0,7.0,7.0,NaN,NaN,5.0,4.0,NaN,11.0,13.0,3.0,4.0,7.0,0.0
2023-10-05 23:00:30-05:00,10.0,7.0,6.0,NaN,NaN,5.0,7.0,NaN,11.0,13.0,2.0,2.0,7.0,0.0


## Descriptive Statistics

In [5]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,2708.000000,2619.000000,2666.000000,1303.000000,2552.000000,2333.000000,2579.000000,2535.000000,2647.000000,2666.000000,2602.00000,2697.000000,2693.000000,2682.000000
mean,8.597858,10.864834,20.539385,13.214121,11.654781,11.763823,9.324932,11.542801,12.012845,13.578395,8.39239,9.943270,10.751207,0.001491
std,3.521273,5.413115,78.506616,9.977442,11.534396,4.506564,4.982347,4.781125,3.958284,7.736178,4.91920,19.195661,6.773158,0.038597
min,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,3.000000,5.000000,4.000000,5.000000,0.00000,0.000000,3.000000,0.000000
25%,6.000000,6.000000,5.000000,9.000000,5.000000,8.000000,5.000000,8.000000,9.000000,9.000000,5.00000,5.000000,7.000000,0.000000
50%,8.000000,10.000000,10.000000,11.000000,7.000000,11.000000,8.000000,10.000000,12.000000,12.000000,7.00000,7.000000,9.000000,0.000000
75%,11.000000,13.000000,23.000000,13.000000,14.000000,16.000000,13.000000,13.000000,15.000000,16.000000,11.00000,12.000000,13.000000,0.000000
max,35.000000,37.000000,2061.000000,78.000000,257.000000,23.000000,32.000000,29.000000,26.000000,77.000000,76.00000,604.000000,66.000000,1.000000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 45%, Missing: 55%, To be checked
Tangara Sensor: TANGARA_1EE6, Data: 89%, Missing: 11%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 81%, Missing: 19%, To be checked
Tangara Sensor: TANGARA_2FF6, Data: 88%, Missing: 12%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_raw, "pm25_raw.csv")